In [59]:
import numpy as np
from keras import layers
from load_data import load
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Conv1D, Add, Reshape
from keras.layers import AveragePooling2D, UpSampling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import keras.backend as K
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2DTranspose
from keras.layers.core import Lambda
from keras.models import Model

K.set_image_data_format('channels_last')
print('Done')

Done


In [2]:
[X_train,Y_train,X_test,Y_test] = load("blast_tab_1hit.out")

Loading data...
58863 sequences were uploaded

Maximum sequence length in is 308
Maximum sequence length out is 308

Converting to one-hot...
Done
Converting to one-hot...
Done


In [3]:
m = X_train.shape[0]
print("There are " + str(m) + " training examples")
print("There are " + str(X_test.shape[0]) + " testing examples")
print("There are " + str(X_train.shape[1]) + " classes: A, C, G, T")
max_length = max(X_train.shape[2],X_test.shape[2])
print("The longest sequence is " + str(max_length) + " nucleotides long")
print(X_train.shape)

There are 55996 training examples
There are 2867 testing examples
There are 4 classes: A, C, G, T
The longest sequence is 308 nucleotides long
(55996, 4, 308, 1)


In [4]:
print('Permuting...')
np.random.seed(0)
rand_perm = np.random.rand(m).argsort()
np.take(X_train,rand_perm,axis=0,out=X_train)
print("finished X")
np.take(Y_train,rand_perm,axis=0,out=Y_train)
print("finished Y")

Permuting...
finished X
finished Y


In [7]:
# Visualize data sets to ensure they appear as anticipated
sample = 300
print(X_train[sample,:,10:30,0])
print(Y_train[sample,:,10:30,0])

[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1.]]
[[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0.]
 [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1.]]


In [130]:
ngf = 16
input_nc = 1
output_nc = 1
n_blocks_gen = 9

# started from: https://blog.sicara.com/keras-generative-adversarial-networks-image-deblurring-45e3ab6977b5
def Model_1(input_shape):
    """Build generator architecture."""
    # Current version : ResNet block
    X_input = Input(input_shape)
    
    # X = ZeroPadding2D((0, 3))(X_input) # allows 'valid' in conv0 to keep length and collapse one-hot

    X = Conv2D(128, (4, 7), strides = (1, 1), padding = 'same', name = 'conv0')(X_input)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)
    
    # X = Conv2D(64, (1, 7), strides = (1, 1), padding = 'same', name = 'conv1')(X)
    # X = BatchNormalization(axis = 3, name = 'bn1')(X)
    # X = Activation('relu')(X)
    
    X = Conv2D(64, (1, 7), strides = (1, 1), padding = 'same', name = 'conv2')(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)
    
    X = Dropout(0.3, name = 'dropout')(X)
   
    X = Conv2D(32, (1, 7), strides = (1, 1), padding = 'same', name = 'conv3')(X)
    X = BatchNormalization(axis = 3, name = 'bn3')(X)
    X = Activation('relu')(X)
    
    # X = Conv2D(4, (1, 1), strides = (1, 1), padding = 'same', data_format = 'channels_first', name = 'conv4')(X)
    # X = Activation('relu')(X)
    
    X = Conv2D(1, (1, 1), strides = (1, 1), padding = 'same', name = 'conv5')(X)
    X = BatchNormalization(axis = 3, name = 'bn5')(X)
    X = Activation('sigmoid')(X)
    
    model = Model(inputs=X_input, outputs=X, name='Model_1')
    return model

In [131]:
myModel = Model_1((4,max_length,1))
print(myModel.summary())
print('Done!')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 4, 308, 1)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 4, 308, 128)       3712      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 4, 308, 128)       512       
_________________________________________________________________
activation_124 (Activation)  (None, 4, 308, 128)       0         
_________________________________________________________________
conv2 (Conv2D)               (None, 4, 308, 64)        57408     
_________________________________________________________________
bn2 (BatchNormalization)     (None, 4, 308, 64)        256       
_________________________________________________________________
activation_125 (Activation)  (None, 4, 308, 64)        0         
__________

In [132]:
myModel.compile(optimizer="Adam", loss="mean_squared_error", metrics = ["accuracy"])
print('Done!')

Done!


In [ ]:
myModel.fit(x = X_train, y = Y_train, epochs = 100, batch_size = 50)

Epoch 1/100
55996/55996 [==============================] - 4869s 87ms/step - loss: 0.0143 - acc: 0.9968
Epoch 2/100
55996/55996 [==============================] - 11671s 208ms/step - loss: 0.0070 - acc: 0.9971
Epoch 3/100
55996/55996 [==============================] - 10830s 193ms/step - loss: 0.0046 - acc: 0.9972
Epoch 4/100
55996/55996 [==============================] - 5463s 98ms/step - loss: 0.0036 - acc: 0.9973
Epoch 5/100
41400/55996 [=====================>........] - ETA: 11:41 - loss: 0.0031 - acc: 0.9973

In [134]:
preds = myModel.predict(x = X_test)

In [135]:
# preds = preds > 0.9
n = 1
samp_n_pred = preds[n,0:4,0:6,0]
samp_n_true = Y_test[n,0:4,0:6,0]
print(samp_n_pred)
print(samp_n_true)


[[0.9382874  0.16544516 0.16041994 0.16773294 0.9271143  0.16600382]
 [0.1559768  0.9371191  0.16428974 0.16269664 0.16381405 0.17209382]
 [0.15969662 0.16143295 0.9323618  0.16832909 0.16696222 0.93291306]
 [0.15192789 0.161154   0.16576166 0.92814827 0.16706608 0.16385233]]
[[1. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]]
